In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import Conv2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation,Flatten,Dropout,Dense,BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras import optimizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

train_dir = "train/"
print(os.listdir("."))
!ls -1 train/ | wc -l

Using TensorFlow backend.


['.config', 'test', 'sample_submission.csv', 'train.csv', 'class-mapping.csv', 'train', 'adc.json']
554656


In [3]:
class_mapping = pd.read_csv( 'class-mapping.csv' )
print(class_mapping.head())
train_csv = pd.read_csv( 'train.csv', header=None, names=['image','class'] )
print(train_csv.head())

   numeric japanese english_translated_by_google_translate
0        2      お菓子                                 Sweets
1       33        肉                                   Meat
2       37       野菜                             Vegetables
3       41        魚                                   Fish
4       38      鍋料理                              Casserole
            image  class
0  1000000965.jpg     40
1  1310005493.jpg     31
2  1050002320.jpg     37
3  1340001721.jpg     33
4  1650001155.jpg      3


In [4]:
batch_size=64
epochs=10
height=299
width=299

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=90,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split=0.30)

train_generator=datagen.flow_from_dataframe(
        dataframe=train_csv,
        directory=train_dir,
        x_col="image",
        y_col="class",
        has_ext=True,                                      
        subset="training",
        batch_size=batch_size,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=(height,width))

valid_generator=datagen.flow_from_dataframe(
        dataframe=train_csv,
        directory=train_dir,
        x_col="image",
        y_col="class",
        has_ext=True,
        subset="validation",
        batch_size=batch_size,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=(height,width))

Found 388260 images belonging to 43 classes.
Found 166396 images belonging to 43 classes.


In [5]:
# Xception CNN Architecture - 1 epoch - 57% accuracy - 4 hours

from keras.applications import *

height, width = 299, 299

pretrained_model = xception.Xception(weights = "imagenet", include_top=False, input_shape = (height, width, 3))

for layer in pretrained_model.layers:
  layer.trainable = False
  
x = pretrained_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(43, activation='softmax')(x)

model = Model(pretrained_model.input, predictions)

model.summary()

83689472/83683744 [==============================] - 4s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________

In [0]:
def top_3_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)
  
model.compile(
    optimizer='nadam',
    loss="categorical_crossentropy",
    metrics=[ "acc", top_3_accuracy ])

In [0]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size

callback_list = [
      ModelCheckpoint('./model.h5', monitor='val_acc',  mode='max', verbose=1, save_best_only=True),
      EarlyStopping(monitor='val_acc', patience=3, verbose=1)
    ]

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=epochs,
    verbose=1,
    callbacks=callback_list)

Epoch 1/10
 588/6066 [=>............................] - ETA: 2:48:58 - loss: 2.4792 - acc: 0.3278 - top_3_accuracy: 0.5657

In [58]:
model.evaluate_generator(generator=valid_generator,steps=STEP_SIZE_VALID,verbose=1)

2599/2599 [==============================] - 2233s 859ms/step


[2.6279029169574124, 0.295842050842383, 0.5208739148166603]

In [59]:
# serialize model to JSON
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
print("Model Saved Successfully")

Model Saved Successfully


In [60]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'model.json'})
uploaded.SetContentFile('model.json')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))
uploaded = drive.CreateFile({'title': 'model.h5'})
uploaded.SetContentFile('model.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1fcbdoTTJhMaLDHKbCJZMnyX4RSJg-puB
Uploaded file with ID 1ruPnFfTBf1Es9yC_G3lBVQdOF9VpNI6-


In [0]:
# AlexNet CNN Archtecture - 1 epoch - 45% accuracy - 2.5 hours

# model = Sequential()

# model.add(Conv2D(96, (11, 11), padding='same', strides=4, input_shape=(height,width,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, (5, 5), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(384, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(384, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(256, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(4096))
# model.add(Activation('relu'))
# model.add(Dropout(0.25))
# model.add(Dense(4096))
# model.add(Activation('relu'))
# model.add(Dense(43, activation='softmax'))

# model.summary()

In [0]:
#
# MobileNet Version1 CNN Architecture - 1 epoch - 55% accuracy - 1.75 hours
#
# from keras.applications import *

# base_model=MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.

# for layer in base_model.layers:
#     layer.trainable=False
    
# x=base_model.output
# x=GlobalAveragePooling2D()(x)
# x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# x=Dense(1024,activation='relu')(x) #dense layer 2
# x=Dense(512,activation='relu')(x) #dense layer 3
# preds=Dense(43,activation='softmax')(x) #final layer with softmax activation

# model=Model(inputs=base_model.input,outputs=preds)

# model.summary()

In [0]:
#
# MobileNet Version2 CNN Architecture - 1 epoch - 57.4% accuracy - 1.75 hours
#
# from keras.applications import *

# base_model=MobileNetV2(weights='imagenet',include_top=False,input_shape=(224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.

# for layer in base_model.layers:
#     layer.trainable=False
    
# x=base_model.output
# x=GlobalAveragePooling2D()(x)
# x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# x=Dense(1024,activation='relu')(x) #dense layer 2
# x=Dense(512,activation='relu')(x) #dense layer 3
# preds=Dense(43,activation='softmax')(x) #final layer with softmax activation

# model=Model(inputs=base_model.input,outputs=preds)

# model.summary()